<a href="https://colab.research.google.com/github/AirNicco8/NLP_Unibo_Project_Work/blob/main/NLI_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Utils

In [1]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 441 kB 31.0 MB/s 
     |████████████████████████████████| 115 kB 66.3 MB/s 
     |████████████████████████████████| 212 kB 69.2 MB/s 
     |████████████████████████████████| 163 kB 44.3 MB/s 
     |████████████████████████████████| 127 kB 48.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [2]:
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 5.0 MB/s 
     |████████████████████████████████| 5.3 MB 55.7 MB/s 
     |████████████████████████████████| 1.3 MB 48.3 MB/s 
     |████████████████████████████████| 7.6 MB 34.8 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=27b6cd2bfd0201f709680c68b5e87e9ec682b90654c4560f694ceb5c21a369a5
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


# Init

In [3]:
import math
from torch.utils.data import DataLoader
from sentence_transformers import models, losses, datasets
from sentence_transformers import LoggingHandler, SentenceTransformer, util, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
import logging
from datetime import datetime
import sys
import os
import gzip
import csv
import random

from sklearn.manifold import TSNE
from matplotlib import pyplot as plt
import seaborn as sns
from pathlib import Path

# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import numpy as np

import torch

def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)

In [4]:
#### Just some code to print debug information to stdout
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])

# Data Preparation

### Download Dataset

In [5]:
!wget 'https://dl.fbaipublicfiles.com/XNLI/XNLI-1.0.zip'

--2022-10-20 15:11:25--  https://dl.fbaipublicfiles.com/XNLI/XNLI-1.0.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.75.142, 172.67.9.4, 104.22.74.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.75.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17865352 (17M) [application/zip]
Saving to: ‘XNLI-1.0.zip’

XNLI-1.0.zip        100%[===================>]  17.04M  6.71MB/s    in 2.5s    

2022-10-20 15:11:29 (6.71 MB/s) - ‘XNLI-1.0.zip’ saved [17865352/17865352]



In [6]:
!unzip './XNLI-1.0.zip'

Archive:  ./XNLI-1.0.zip
   creating: XNLI-1.0/
  inflating: XNLI-1.0/.DS_Store      
   creating: __MACOSX/
   creating: __MACOSX/XNLI-1.0/
  inflating: __MACOSX/XNLI-1.0/._.DS_Store  
  inflating: XNLI-1.0/xnli.dev.tsv   
  inflating: __MACOSX/XNLI-1.0/._xnli.dev.tsv  
  inflating: XNLI-1.0/xnli.dev.jsonl  
  inflating: XNLI-1.0/README.md      
  inflating: __MACOSX/XNLI-1.0/._README.md  
  inflating: XNLI-1.0/xnli.test.jsonl  
  inflating: XNLI-1.0/xnli.test.tsv  
  inflating: __MACOSX/XNLI-1.0/._xnli.test.tsv  
  inflating: __MACOSX/._XNLI-1.0     


### Data Pre Processing

In [7]:
df = pd.read_table('/content/XNLI-1.0/xnli.test.tsv', header=0)
df = df.loc[df['language']=='de']

In [8]:
todrop = set(df.columns.values)-set(['gold_label','sentence1', 'sentence2'])

In [9]:
df = df.drop(columns=todrop) # keep onlt the columns we need

In [10]:
sen1_col = df.columns.values[1] 
sen2_col = df.columns.values[2] 
label_col = df.columns.values[0]

In [11]:
labels = np.unique(df.gold_label)
class_dict={}
for i,l in zip(range(len(labels)), labels):
  class_dict[l]=i # create a dict to map 'neutral', 'entailment', 'contradiction' to integers

In [12]:
class_dict

{'contradiction': 0, 'entailment': 1, 'neutral': 2}

In [13]:
df.gold_label = df.gold_label.map(class_dict) # apply dict
df

,gold_label,sentence1,sentence2
10020,0,"Nun, daran dachte ich nicht einmal, aber ich w...",Ich habe nicht wieder mit ihm gesprochen.
10021,1,"Nun, daran dachte ich nicht einmal, aber ich w...","Ich war so mitgenommen, dass ich einfach wiede..."
10022,2,"Nun, daran dachte ich nicht einmal, aber ich w...",Wir hatten ein tolles Gespräch.
10023,2,"„Und ich dachte, das wäre ein Privileg, und da...","Mir war nicht bewusst, dass ich nicht die einz..."
10024,1,"„Und ich dachte, das wäre ein Privileg, und da...","Ich hatte den Eindruck, dass ich im AFCF Air F..."
...,...,...,...
15025,1,Davidson sollte nicht die Aussprache von Scone...,"Davidson sollte nicht so reden, dass Knochen u..."
15026,0,Davidson sollte nicht die Aussprache von Scone...,"Es wäre besser, wenn Davidson sich mit den Wör..."
15027,2,Der durchschnittliche Roman von 200.000 Wörter...,Ein 200.000 Wort Roman für 25 Dollar ist ein f...
15028,0,Der durchschnittliche Roman von 200.000 Wörter...,Ein 200.000 Wort Roman für $25 sind 4.000 Wört...


# Parallel Dataset

In [14]:
train_samples = []

x1_train = df[sen1_col].values.tolist()
x2_train = df[sen2_col].values.tolist()
y_train = df[label_col].values.tolist()

for i,j,k in zip(x1_train, x2_train, y_train):
  train_samples.append(InputExample(texts=[i, j], label=k))

In [15]:
from datasets import load_dataset # STS dataset for evaluation

dev_set = load_dataset("stsb_multi_mt",'de',split="dev")
test_set = load_dataset("stsb_multi_mt",'de',split="test")

Generating train split:   0%|          | 0/5749 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1379 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/1500 [00:00<?, ? examples/s]

Dataset stsb_multi_mt downloaded and prepared to /root/.cache/huggingface/datasets/stsb_multi_mt/de/1.0.0/a5d260e4b7aa82d1ab7379523a005a366d9b124c76a5a5cf0c4c5365458b0ba9. Subsequent calls will reuse this data.


In [16]:
dev_samples = []

for i in dev_set:
  dev_samples.append(InputExample(texts=[i['sentence1'], i['sentence2']], label=float(i['similarity_score']) / 5.0)) # normalize score

In [17]:
evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples, name='sts-dev')

In [18]:
test_samples = []

for i in test_set:
  test_samples.append(InputExample(texts=[i['sentence1'], i['sentence2']], label=float(i['similarity_score']) / 5.0)) # normalize score

# Multilanguage model

##Params

In [19]:
st_model = "airnicco8/xlm-roberta-en-it-de" #@param ["airnicco8/xlm-roberta-en-it-de","airnicco8/xlm-roberta-de"]
train_batch_size = 16 #@param ["8", "16", "32", "64", "128", "256", "512"]
num_epochs = 1 #@param ["1", "8", "16", "32", "64", "128", "256", "512"]

In [30]:
!mkdir output

In [28]:
model_save_path = './output/'

## Multilanguage Model Training

In [21]:
model = SentenceTransformer(st_model)

Downloading:   0%|          | 0.00/1.49k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/698 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/124 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/280 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/473 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [22]:
train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=train_batch_size)
train_loss = losses.SoftmaxLoss(model=model, sentence_embedding_dimension=model.get_sentence_embedding_dimension(), num_labels=len(class_dict))

warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1) #10% of train data for warm-up

In [23]:
model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=evaluator,
          epochs=num_epochs,
          evaluation_steps=1000,
          warmup_steps=warmup_steps)

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/314 [00:00<?, ?it/s]

## Multilanguage Model Evaluation

In [31]:
test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_samples, name='sts-test')
test_evaluator(model, output_path=model_save_path)

0.6412974138693235

In [32]:
test_df = pd.read_csv('/content/output/similarity_evaluation_sts-test_results.csv')

In [33]:
test_df

,epoch,steps,cosine_pearson,cosine_spearman,euclidean_pearson,euclidean_spearman,manhattan_pearson,manhattan_spearman,dot_pearson,dot_spearman
0,-1,-1,0.544815,0.641297,0.560545,0.621046,0.565209,0.623012,0.405455,0.431346
